# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-217c3e11ac-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("Place of Birth, Death, and Burial") 

Consider the following exploratory information need:

> You want to visit cities connected to famous writers and poets, and you are deciding wether to visit France or Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q183`     | Germany       | node |
| `wd:Q142`     | France        | node |
| `wd:Q90`      | Paris         | node |
| `wd:Q49757`   | Poet          | node |
| `wd:Q36180`   | Writer        | node |
| `wd:Q501`     | Charles Baudelaire  | node      |
| `wd:Q272208`  | Montparnasse Cemetery       | node |


Also consider

```
?p wdt:P27 wd:Q142  . 
?p wdt:P106 wd:Q36180  . 
```

is the BGP to retrieve all **French writers**

## Workload Goals

1. Identify the BGP that connect people to their place of birth or place, death, or burial

2. Identify the BGP to obtain the country in which a place is located

3. How many poets and writers  have a place of birth, death, or burial in Germany and France?

4. Analyze cities across the two countries
 
   4.1 Is there any poet for which the birth place and the place of burial are located in the same city either in Germany or France?
   
   4.2 Which cities host the place of birth of the larger number of poets or writers across the two countries?
   
   4.3 What are the top 3 cities in each country that you could visit? Based on what criteria?


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P27 wd:Q142  . 
?p wdt:P106 wd:Q36180  . 
} 
GROUP BY ?cult  ?arch
"""

print("Predicates")
run_query(queryString)

Predicates
[('callret-0', '14400')]


1

In [ ]:
#I identify the BGP for place of burial througth the Montparnasse Cemetery

In [26]:
queryString = """
SELECT ?q ?d ?qname ?dname
WHERE { 
 ?q ?d wd:Q272208.
 ?q <http://schema.org/name> ?qname.
 ?d <http://schema.org/name> ?dname
} 
LIMIT 10
"""

print("Predicates")
run_query(queryString)

Predicates
[('q', 'http://www.wikidata.org/entity/Q310715'), ('d', 'http://www.wikidata.org/prop/direct/P119'), ('qname', 'Henri Fantin-Latour'), ('dname', 'place of burial')]
[('q', 'http://www.wikidata.org/entity/Q15729827'), ('d', 'http://www.wikidata.org/prop/direct/P119'), ('qname', 'Henri Plon'), ('dname', 'place of burial')]
[('q', 'http://www.wikidata.org/entity/Q970511'), ('d', 'http://www.wikidata.org/prop/direct/P119'), ('qname', 'Charles Meynier'), ('dname', 'place of burial')]
[('q', 'http://www.wikidata.org/entity/Q16889566'), ('d', 'http://www.wikidata.org/prop/direct/P119'), ('qname', 'Henri Marie Bouley'), ('dname', 'place of burial')]
[('q', 'http://www.wikidata.org/entity/Q313556'), ('d', 'http://www.wikidata.org/prop/direct/P119'), ('qname', 'Charles Garnier'), ('dname', 'place of burial')]
[('q', 'http://www.wikidata.org/entity/Q62990549'), ('d', 'http://www.wikidata.org/prop/direct/P119'), ('qname', 'Jacques Libman'), ('dname', 'place of burial')]
[('q', 'http://w

10

In [ ]:
#so the BGP for place of burial is P119

In [ ]:
#i identify the bgp for place of birth or death for Henri Fantin-Latour 

In [28]:

queryString = """
SELECT DISTINCT ?pname ?a
WHERE { 

wd:Q310715 ?a ?b.



?a <http://schema.org/name> ?pname
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Portuguese National Library ID'), ('a', 'http://www.wikidata.org/prop/direct/P1005')]
[('pname', 'Nationale Thesaurus voor Auteurs ID'), ('a', 'http://www.wikidata.org/prop/direct/P1006')]
[('pname', 'occupation'), ('a', 'http://www.wikidata.org/prop/direct/P106')]
[('pname', 'student of'), ('a', 'http://www.wikidata.org/prop/direct/P1066')]
[('pname', 'place of burial'), ('a', 'http://www.wikidata.org/prop/direct/P119')]
[('pname', 'NUKAT ID'), ('a', 'http://www.wikidata.org/prop/direct/P1207')]
[('pname', 'KulturNav-ID'), ('a', 'http://www.wikidata.org/prop/direct/P1248')]
[('pname', 'NNDB people ID'), ('a', 'http://www.wikidata.org/prop/direct/P1263')]
[('pname', 'CANTIC ID'), ('a', 'http://www.wikidata.org/prop/direct/P1273')]
[('pname', 'CONOR.SI ID'), ('a', 'http://www.wikidata.org/prop/direct/P1280')]
[('pname', 'Gran Enciclopèdia Catalana ID'), ('a', 'http://www.wikidata.org/prop/direct/P1296')]
[('pname', 'NLA Trove ID'), ('a', 'http://www.wikidata.org/prop

50

In [ ]:
#the BGP for place of birth is P19 and for place of death is P20

In [ ]:
#I Identify the BGP to obtain the country in which a place is located using the place paris



In [18]:
 
queryString = """
SELECT DISTINCT ?pname ?relation 
WHERE { 

wd:Q90 ?relation ?q.

?relation <http://schema.org/name> ?pname
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Vatican Library ID (former scheme)'), ('relation', 'http://www.wikidata.org/prop/direct/P1017')]
[('pname', 'Dewey Decimal Classification'), ('relation', 'http://www.wikidata.org/prop/direct/P1036')]
[('pname', 'population'), ('relation', 'http://www.wikidata.org/prop/direct/P1082')]
[('pname', "topic's main Wikimedia portal"), ('relation', 'http://www.wikidata.org/prop/direct/P1151')]
[('pname', 'U.S. National Archives Identifier'), ('relation', 'http://www.wikidata.org/prop/direct/P1225')]
[('pname', 'OmegaWiki Defined Meaning'), ('relation', 'http://www.wikidata.org/prop/direct/P1245')]
[('pname', 'Gran Enciclopèdia Catalana ID'), ('relation', 'http://www.wikidata.org/prop/direct/P1296')]
[('pname', 'located in the administrative territorial entity'), ('relation', 'http://www.wikidata.org/prop/direct/P131')]
[('pname', 'office held by head of government'), ('relation', 'http://www.wikidata.org/prop/direct/P1313')]
[('pname', 'described by source'), ('relation', '

50

In [14]:
#i obtained the BGP for country wich is P17

In [ ]:
#How many poets and writers have a place of birth, death, or burial in Germany and France?

In [9]:
#att:sbagliato quì ripete la somma
#here i find the writer who have place of death birth or burial in france 
queryString = """

SELECT DISTINCT COUNT(?pname) 
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180  .

?nameF <http://schema.org/name> ?pname.
} 

UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P20 ?place . 
?nameF wdt:P106 wd:Q36180  .

?nameF <http://schema.org/name> ?pname.
} 

UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P119 ?place . 
?nameF wdt:P106 wd:Q36180  .

?nameF <http://schema.org/name> ?pname.
} 
}

"""

print("Results")
run_query(queryString)

Results
[('callret-0', '20641')]


1

In [4]:

#here i find the writer who have place of death birth or burial in france 
queryString = """

SELECT DISTINCT ?pname COUNT(?nameF)
WHERE {

?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q36180  .

?nameF <http://schema.org/name> ?pname.


}LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('pname', 'Enguerrand Homps'), ('callret-1', '1')]
[('pname', 'Gustave Geffroy'), ('callret-1', '1')]
[('pname', 'Jeanne Lapauze'), ('callret-1', '2')]
[('pname', 'Charles, Duke of Orléans'), ('callret-1', '1')]
[('pname', 'Georges Darien'), ('callret-1', '1')]
[('pname', 'Nicolas Artaud'), ('callret-1', '1')]
[('pname', 'Boris Ûlianovič Poplavskij'), ('callret-1', '1')]
[('pname', 'Louis Bouilhet'), ('callret-1', '1')]
[('pname', 'Edmée de La Rochefoucauld'), ('callret-1', '1')]
[('pname', 'Frédéric Soulié'), ('callret-1', '1')]


10

In [21]:
#SBAGLIATA GUARDA LA PROSSIMA
#here i find the poet or writer who have place of death birt or burial in france 
queryString = """

SELECT DISTINCT COUNT(?nameF) AS ?number
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}

}

"""

print("Results")
run_query(queryString)

Results
[('number', '2350')]


1

In [22]:
#SBAGLIATA GUARDA LA PROSSIMA
#here i find the poet or writer who have place of death birt or burial in france or germany wd:Q183
queryString = """

SELECT DISTINCT COUNT(?nameF) AS ?number
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}
UNION
{
?place wdt:P17 wd:Q183.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}

}

"""

print("Results")
run_query(queryString)

Results
[('number', '3629')]


1

In [34]:
#SBAGLIATA RIFARE
#here i find the number of poet or writer who have place of death birt or burial in france or germany wd:Q183
queryString = """
SELECT DISTINCT COUNT(?nameF) AS ?numero
{
SELECT DISTINCT ?nameF 
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}
UNION
{
?place wdt:P17 wd:Q183.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P119/wdt:P19*/wdt:P20* ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}

}
}
"""

print("Results")
run_query(queryString)

Results
[('numero', '2940')]


1

In [3]:
##here i find the number of poet or writer who have place of death birth or burial in france or germany wd:Q183
queryString = """
SELECT DISTINCT COUNT(distinct ?nameF) AS ?numero
{
SELECT DISTINCT ?nameF 
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P119 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P119 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}
UNION
{
?place wdt:P17 wd:Q183.
?nameF wdt:P119 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P119 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}








UNION 
{
?place wdt:P17 wd:Q142.
?nameF wdt:P20 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P20 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}
UNION
{
?place wdt:P17 wd:Q183.
?nameF wdt:P20 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P20 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}





UNION


{
?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}
UNION
{
?place wdt:P17 wd:Q183.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}




}
}
"""

print("Results")
run_query(queryString)

Results
[('numero', '42514')]


1

In [ ]:
# Is there any poet for which the birth place and the place of burial are located in the same city either in Germany or France?

In [29]:
#here i identify a city trougth the relationship with Henri Fantin-Latour 
queryString = """
SELECT DISTINCT ?pname ?a ?bname ?b
WHERE { 

wd:Q310715 ?a ?b.


?b <http://schema.org/name> ?bname.
?a <http://schema.org/name> ?pname.
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pname', 'occupation'), ('a', 'http://www.wikidata.org/prop/direct/P106'), ('bname', 'painter'), ('b', 'http://www.wikidata.org/entity/Q1028181')]
[('pname', 'occupation'), ('a', 'http://www.wikidata.org/prop/direct/P106'), ('bname', 'printmaker'), ('b', 'http://www.wikidata.org/entity/Q11569986')]
[('pname', 'occupation'), ('a', 'http://www.wikidata.org/prop/direct/P106'), ('bname', 'graphic artist'), ('b', 'http://www.wikidata.org/entity/Q1925963')]
[('pname', 'occupation'), ('a', 'http://www.wikidata.org/prop/direct/P106'), ('bname', 'lithographer'), ('b', 'http://www.wikidata.org/entity/Q16947657')]
[('pname', 'occupation'), ('a', 'http://www.wikidata.org/prop/direct/P106'), ('bname', 'botanical illustrator'), ('b', 'http://www.wikidata.org/entity/Q3148760')]
[('pname', 'student of'), ('a', 'http://www.wikidata.org/prop/direct/P1066'), ('bname', 'Horace Lecoq de Boisbaudran'), ('b', 'http://www.wikidata.org/entity/Q3140465')]
[('pname', 'student of'), ('a', 'http://www.wi

50

In [31]:
#here i get all the relationship with greenoble
queryString = """
SELECT DISTINCT ?pname ?a ?bname ?b
WHERE { 

wd:Q1289 ?a ?b.


?b <http://schema.org/name> ?bname.
?a <http://schema.org/name> ?pname.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pname', "topic's main Wikimedia portal"), ('a', 'http://www.wikidata.org/prop/direct/P1151'), ('bname', 'Portal:Grenoble Métropole'), ('b', 'http://www.wikidata.org/entity/Q15649944')]
[('pname', 'located in the administrative territorial entity'), ('a', 'http://www.wikidata.org/prop/direct/P131'), ('bname', 'Isère'), ('b', 'http://www.wikidata.org/entity/Q12559')]
[('pname', 'located in the administrative territorial entity'), ('a', 'http://www.wikidata.org/prop/direct/P131'), ('bname', 'arrondissement of Grenoble'), ('b', 'http://www.wikidata.org/entity/Q132224')]
[('pname', 'located in the administrative territorial entity'), ('a', 'http://www.wikidata.org/prop/direct/P131'), ('bname', 'canton of Grenoble-5'), ('b', 'http://www.wikidata.org/entity/Q1725421')]
[('pname', 'located in the administrative territorial entity'), ('a', 'http://www.wikidata.org/prop/direct/P131'), ('bname', 'canton of Grenoble-6'), ('b', 'http://www.wikidata.org/entity/Q1725423')]
[('pname', 'offi

71

In [32]:
#here i get all the relationship with big city Q1549591 found in the query above
queryString = """
SELECT DISTINCT ?pname ?a ?bname ?b
WHERE { 

wd:Q1549591 ?a ?b.


?b <http://schema.org/name> ?bname.
?a <http://schema.org/name> ?pname.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pname', 'next lower rank'), ('a', 'http://www.wikidata.org/prop/direct/P3729'), ('bname', 'town'), ('b', 'http://www.wikidata.org/entity/Q3957')]
[('pname', 'has quality'), ('a', 'http://www.wikidata.org/prop/direct/P1552'), ('bname', 'minimum population'), ('b', 'http://www.wikidata.org/entity/Q51706937')]
[('pname', 'subclass of'), ('a', 'http://www.wikidata.org/prop/direct/P279'), ('bname', 'city'), ('b', 'http://www.wikidata.org/entity/Q515')]
[('pname', "topic's main category"), ('a', 'http://www.wikidata.org/prop/direct/P910'), ('bname', 'Category:Big cities'), ('b', 'http://www.wikidata.org/entity/Q9823231')]
[('pname', 'next higher rank'), ('a', 'http://www.wikidata.org/prop/direct/P3730'), ('bname', 'million city'), ('b', 'http://www.wikidata.org/entity/Q1637706')]


5

In [ ]:
#now i get the iri of city Q515

In [33]:
# Is there any poet for which the birth place and the place of burial are located in the same city either in Germany or France

queryString = """
SELECT DISTINCT ?burial ?birth ?bname ?buname
WHERE {

{
?burial wdt:P17 wd:Q142.
?nameF wdt:P119 ?burial. 
?birth wdt:P17 wd:Q142.
?nameF wdt:P19 ?birth. 
?nameF wdt:P106 wd:Q49757 .

?burial <http://schema.org/name> ?buname.
?birth <http://schema.org/name> ?bname.

}

UNION
{



?burial wdt:P17 wd:Q183.
?nameF wdt:P119 ?burial. 
?birth wdt:P17 wd:Q183.
?nameF wdt:P19 ?birth. 
?nameF wdt:P106 wd:Q49757 .

?burial <http://schema.org/name> ?buname.
?birth <http://schema.org/name> ?bname.




}

FILTER(?birth=?burial)
}

"""

print("Results")
run_query(queryString)











Results
[('burial', 'http://www.wikidata.org/entity/Q1170765'), ('birth', 'http://www.wikidata.org/entity/Q1170765'), ('bname', 'Véretz'), ('buname', 'Véretz')]
[('burial', 'http://www.wikidata.org/entity/Q275438'), ('birth', 'http://www.wikidata.org/entity/Q275438'), ('bname', 'Argenton-sur-Creuse'), ('buname', 'Argenton-sur-Creuse')]
[('burial', 'http://www.wikidata.org/entity/Q815838'), ('birth', 'http://www.wikidata.org/entity/Q815838'), ('bname', 'Belle Île'), ('buname', 'Belle Île')]
[('burial', 'http://www.wikidata.org/entity/Q202368'), ('birth', 'http://www.wikidata.org/entity/Q202368'), ('bname', 'Morlaix'), ('buname', 'Morlaix')]
[('burial', 'http://www.wikidata.org/entity/Q209093'), ('birth', 'http://www.wikidata.org/entity/Q209093'), ('bname', 'Bar-sur-Aube'), ('buname', 'Bar-sur-Aube')]
[('burial', 'http://www.wikidata.org/entity/Q209776'), ('birth', 'http://www.wikidata.org/entity/Q209776'), ('bname', 'Coadout'), ('buname', 'Coadout')]
[('burial', 'http://www.wikidata.org

22

In [ ]:
#i found that there exist some poet with the same place of burial and birth in most cases this place is a city so the answer is yes

In [6]:
#Which cities host the place of birth of the larger number of poets or writers across the two countries

queryString = """
SELECT DISTINCT COUNT(distinct ?nameF) AS ?numero ?place AS ?city
{
SELECT DISTINCT ?nameF ?place
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}
UNION
{
?place wdt:P17 wd:Q183.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P19 ?place .
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}

}
}GROUP BY ?place
ORDER BY DESC(?numero)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('numero', '2482'), ('city', 'http://www.wikidata.org/entity/Q90')]
[('numero', '1904'), ('city', 'http://www.wikidata.org/entity/Q64')]
[('numero', '846'), ('city', 'http://www.wikidata.org/entity/Q1055')]
[('numero', '771'), ('city', 'http://www.wikidata.org/entity/Q1726')]
[('numero', '458'), ('city', 'http://www.wikidata.org/entity/Q2079')]
[('numero', '436'), ('city', 'http://www.wikidata.org/entity/Q365')]
[('numero', '420'), ('city', 'http://www.wikidata.org/entity/Q1794')]
[('numero', '372'), ('city', 'http://www.wikidata.org/entity/Q1022')]
[('numero', '365'), ('city', 'http://www.wikidata.org/entity/Q1731')]
[('numero', '310'), ('city', 'http://www.wikidata.org/entity/Q1715')]
[('numero', '255'), ('city', 'http://www.wikidata.org/entity/Q456')]
[('numero', '255'), ('city', 'http://www.wikidata.org/entity/Q23482')]
[('numero', '254'), ('city', 'http://www.wikidata.org/entity/Q1718')]
[('numero', '231'), ('city', 'http://www.wikidata.org/entity/Q2090')]
[('numero', '18

20

In [ ]:
#What are the top 3 cities in each country that you could visit? Based on what criteria? 

In [43]:
#based on the writers/poets who where born in these city i found the top 3 city in france:

queryString = """
SELECT DISTINCT COUNT(?nameF) AS ?numero ?place AS ?city
{
SELECT DISTINCT ?nameF ?place
WHERE {
{
?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q142.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}


}
}GROUP BY ?place
ORDER BY DESC(?numero)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('numero', '2482'), ('city', 'http://www.wikidata.org/entity/Q90')]
[('numero', '255'), ('city', 'http://www.wikidata.org/entity/Q23482')]
[('numero', '255'), ('city', 'http://www.wikidata.org/entity/Q456')]


3

In [45]:
#based on the writers/poets who where born in these city i found the top 3 city in Germany:
queryString = """
SELECT DISTINCT COUNT(?nameF) AS ?numero ?place AS ?city
{
SELECT DISTINCT ?nameF ?place
WHERE {

{
?place wdt:P17 wd:Q183.
?nameF wdt:P19 ?place . 
?nameF wdt:P106 wd:Q36180 .

?nameF <http://schema.org/name> ?pname.


}
UNION
{

?place wdt:P17 wd:Q183.
?nameF wdt:P19 ?place .
?nameF wdt:P106 wd:Q49757 .

?nameF <http://schema.org/name> ?pname.

}

}
}GROUP BY ?place
ORDER BY DESC(?numero)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('numero', '1904'), ('city', 'http://www.wikidata.org/entity/Q64')]
[('numero', '846'), ('city', 'http://www.wikidata.org/entity/Q1055')]
[('numero', '771'), ('city', 'http://www.wikidata.org/entity/Q1726')]


3